### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*10

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'
print males.length(), 'elements'
print fem.length(), 'elements'


Data with 40 male and female speakers
14508 elements
14451 elements


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model configuration

In [4]:
N = 256
max_pool = 256
batch_size = 64
learning_rate = 0.001

config_model = {}
config_model["type"] = "pretraining"

config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size

config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["window"] = 1024

config_model["smooth_size"] = 20

config_model["alpha"] = learning_rate
config_model["reg"] = 1e-4
config_model["beta"] = 0.1
config_model["rho"] = 0.01



# Model creation

In [ ]:
from models.adapt import Adapt

adapt_model = Adapt(config_model=config_model, pretraining=True, folder='pretraining')
print 'Total name :' 
print adapt_model.runID
adapt_model.tensorboard_init()

ID : AdaptiveNet-proud-king-5674
[<tf.Variable 'conv/W:0' shape=(1, 1024, 1, 256) dtype=float32_ref>, <tf.Variable 'smooth/smoothing_filter:0' shape=(1, 20, 256, 256) dtype=float32_ref>]
Total name :
AdaptiveNet-proud-king-5674-N=256--alpha=0.001--batch_size=1--beta=0.1--chunk_size=5120--maxpool=256--reg=0.0001--rho=0.01--smooth_size=20--type=pretraining--window=1024-


# Pretraining the model 

In [ ]:
nb_iterations = 1000

#initialize the model
adapt_model.init()

X_in, X_mix, Ind = mixed_data.get_batch(batch_size)

for i in range(nb_iterations):
    c = adapt_model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        adapt_model.save(i)

Step # 0  loss= 0.370085
DAS model saved at iteration number  0  with cost =  0.370085
Step # 1  loss= 0.367104
Step # 2  loss= 0.36218
Step # 3  loss= 0.350624
Step # 4  loss= 0.326034
Step # 5  loss= 0.282237
Step # 6  loss= 0.222926
Step # 7  loss= 0.167387
Step # 8  loss= 0.14899
Step # 9  loss= 0.158843
Step # 10  loss= 0.15269
Step # 11  loss= 0.129924
Step # 12  loss= 0.0996907
Step # 13  loss= 0.0868637
Step # 14  loss= 0.083717
Step # 15  loss= 0.0782587
Step # 16  loss= 0.0724958
Step # 17  loss= 0.0624649
Step # 18  loss= 0.0518768
Step # 19  loss= 0.0430164
Step # 20  loss= 0.0418889
DAS model saved at iteration number  20  with cost =  0.0418889
Step # 21  loss= 0.0433734
Step # 22  loss= 0.0411686
Step # 23  loss= 0.0412804
Step # 24  loss= 0.0359924
Step # 25  loss= 0.029834
Step # 26  loss= 0.0282409
Step # 27  loss= 0.0290834
Step # 28  loss= 0.0286572
Step # 29  loss= 0.0262399
Step # 30  loss= 0.0253854
Step # 31  loss= 0.0249774
Step # 32  loss= 0.023836
Step # 33  

Step # 266  loss= 0.0166891
Step # 267  loss= 0.0161661
Step # 268  loss= 0.0164999
Step # 269  loss= 0.0163289
Step # 270  loss= 0.0158938
Step # 271  loss= 0.0162957
Step # 272  loss= 0.0165982
Step # 273  loss= 0.0166477
Step # 274  loss= 0.016577
Step # 275  loss= 0.0161676
Step # 276  loss= 0.0161101
Step # 277  loss= 0.0164753
Step # 278  loss= 0.0165792
Step # 279  loss= 0.01633
Step # 280  loss= 0.0162093
DAS model saved at iteration number  280  with cost =  0.0162093
Step # 281  loss= 0.0170727
Step # 282  loss= 0.016727
Step # 283  loss= 0.0161418
Step # 284  loss= 0.0157584
Step # 285  loss= 0.0161333
Step # 286  loss= 0.0162725
Step # 287  loss= 0.0165533
Step # 288  loss= 0.0163339
Step # 289  loss= 0.0166108
Step # 290  loss= 0.0166441
Step # 291  loss= 0.0163374
Step # 292  loss= 0.0156264
Step # 293  loss= 0.0159624
Step # 294  loss= 0.0160928
Step # 295  loss= 0.0161203
Step # 296  loss= 0.0161716
Step # 297  loss= 0.0164156
Step # 298  loss= 0.0159199
Step # 299  los